In [1]:
!pip install openimages

In [2]:
import openimages

In [3]:
from openimages.download import download_dataset
download_dataset("./", ["Human head"], annotation_format="darknet", limit=2000)

2025-06-08  16:46:11 INFO Downloading 2000 train images for class 'human head'
100%|██████████| 2000/2000 [01:19<00:00, 25.20it/s]
2025-06-08  16:47:30 INFO Creating 2000 train annotations (darknet) for class 'human head'
100%|██████████| 2000/2000 [00:00<00:00, 19332.73it/s]


{'human head': {'images_dir': './human head/images',
  'annotations_dir': './human head/darknet'}}

In [4]:
!cat /content/human\ head/darknet/000004f4400f6ec5.txt

cat: '/content/human head/darknet/000004f4400f6ec5.txt': No such file or directory


In [5]:
!pip install ultralytics > /dev/null 2>&1

In [6]:
nc =  1
names = ["Human head"]

with open("data.yaml", "w") as f:
    f.write(f"train: ../train/images\nval: ../valid/images\n\nnc: {nc}\nnames: {names}")

In [7]:
!cat data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['Human head']

In [8]:
!mv human\ head/darknet human\ head/labels

In [9]:
import os
import random
import shutil
import yaml

DATASET_DIR = 'human head'
SAVE_SPLIT_DIR = './'

SIZE_VALID = 0.2

In [13]:
path_train = os.path.join(SAVE_SPLIT_DIR, 'train')
if not os.path.exists(path_train):
    os.mkdir(path_train)
    os.mkdir(path_train_images)
    os.mkdir(path_train_labels)
    os.mkdir(path_valid)
    os.mkdir(path_valid_images)
    os.mkdir(path_valid_labels)
path_train_images = os.path.join(path_train, 'images')
path_train_labels = os.path.join(path_train, 'labels')
path_valid = os.path.join(SAVE_SPLIT_DIR, 'valid')
path_valid_images = os.path.join(path_valid, 'images')
path_valid_labels = os.path.join(path_valid, 'labels')
    

In [14]:
path_images = os.path.join(DATASET_DIR, 'images')
path_labels = os.path.join(DATASET_DIR, 'labels')

In [15]:
images = os.listdir(path_images)

valid_images = random.sample(images, int(len(images) * SIZE_VALID))
print(f'Size valid = {len(valid_images)}')
for filename in images:
    only_filename = filename[:-4]

    if filename in valid_images:
        shutil.copyfile(
            os.path.join(path_images, filename),
            os.path.join(path_valid_images, filename),
        )
        shutil.copyfile(
            os.path.join(path_labels, f'{only_filename}.txt'),
            os.path.join(path_valid_labels, f'{only_filename}.txt'),
        )
    else:
        shutil.copyfile(
            os.path.join(path_images, filename),
            os.path.join(path_train_images, filename),
        )
        shutil.copyfile(
            os.path.join(path_labels, f'{only_filename}.txt'),
            os.path.join(path_train_labels, f'{only_filename}.txt'),
        )

Size valid = 400


In [16]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 22.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.6/341.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 11.5 MB/s eta 0:00:00


In [22]:
import os
WANDB_API_KEY = 'acf1d7088c7b9d4ddc5d01f32602a83e5c16e435'

In [23]:
from ultralytics import settings
settings.update({"wandb": True})
import wandb

# Initialize your Weights & Biases environment
wandb.login(key=WANDB_API_KEY, force=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/andrey/.netrc
wandb: Currently logged in as: aerrries (aerrries-hse) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [27]:
import torch
import ultralytics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_n = ultralytics.YOLO('yolo11n.yaml').to(device)

res_n = model_n.train(data='data.yaml', pretrained=True, epochs=150, batch = 16, cache=False, project = "head_detection", name = "yolo11n")

engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=head_detection, rect=False, resume=False, retina_masks=False, save

train: Scanning /home/andrey/projects/lightsb-service/experiments/train/labels.cache... 1927 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1927/1927 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1563.1±212.1 MB/s, size: 348.6 KB)


val: Scanning /home/andrey/projects/lightsb-service/experiments/valid/labels.cache... 727 images, 0 backgrounds, 0 corrupt: 100%|██████████| 727/727 [00:00<?, ?it/s]


Plotting labels to head_detection/yolo11n2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to head_detection/yolo11n2
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        727       1768      0.795      0.755      0.795      0.544



150 epochs completed in 0.514 hours.
Optimizer stripped from head_detection/yolo11n2/weights/last.pt, 5.5MB
Optimizer stripped from head_detection/yolo11n2/weights/best.pt, 5.5MB

Validating head_detection/yolo11n2/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7930MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:01<00:00, 12.95it/s]


                   all        727       1768      0.795      0.753      0.795      0.544
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to head_detection/yolo11n2


lr/pg0,███▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
lr/pg1,▃████▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▆██▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁
metrics/mAP50(B),▁▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
metrics/mAP50-95(B),▁▂▁▃▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
metrics/precision(B),▁▂▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇████████████
metrics/recall(B),▁▂▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇█▇██████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁


In [24]:
import torch
import ultralytics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ultralytics.YOLO('yolo11s.yaml').to(device)

res = model.train(data='data.yaml', pretrained=True, epochs=150, batch = 16, cache=False, project = "head_detection", name = "yolo11s")

engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=head_detection, rect=False, resume=False, retina_masks=False, save=

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6534.3±2244.5 MB/s, size: 442.9 KB)


train: Scanning /home/andrey/projects/lightsb-service/experiments/train/labels... 1927 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1927/1927 [00:00<00:00, 5359.04it/s]

train: New cache created: /home/andrey/projects/lightsb-service/experiments/train/labels.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2720.0±1222.1 MB/s, size: 348.6 KB)


...

                   all        727       1768      0.852      0.801      0.855      0.621



150 epochs completed in 0.926 hours.
Optimizer stripped from head_detection/yolo11s/weights/last.pt, 19.2MB
Optimizer stripped from head_detection/yolo11s/weights/best.pt, 19.2MB

Validating head_detection/yolo11s/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7930MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00,  8.87it/s]


                   all        727       1768      0.852      0.803      0.856      0.621
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to head_detection/yolo11s


lr/pg0,███▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▁▁
lr/pg1,▃█████▇▇▇▇▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁
lr/pg2,▆███▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▃▂▂▂▂▂▂▁▁▁▁
metrics/mAP50(B),▁▁▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█████████████
metrics/mAP50-95(B),▁▂▂▃▃▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████
metrics/precision(B),▁▄▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇████████
metrics/recall(B),▁▄▄▅▄▅▆▆▆▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
